In [1]:
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pandas as pd
import re

from lib import SQLiteConnection
from lib import get_info_video_judobase
from lib import load_chromedriver

In [2]:
import traceback

In [71]:
df = df.rename(columns={'action2': 'action_detail'})

### Battles Estrella

In [65]:
# dataframe containing battles info
df = conn.as_pandas('select * from estre')

In [67]:
# ver ejemplo
df.head(5)

,opponent,you,action,action_detail,time,url_youtube,event
0,PARK Da Sol,1,Shido,Shido / False-Attack,00:15,https://judobase.ijf.org/#/competition/contest...,Grand Slam Osaka 2018
1,PARK Da Sol,0,Shido,Shido / False-Attack,01:29,https://judobase.ijf.org/#/competition/contest...,Grand Slam Osaka 2018
2,PARK Da Sol,1,Shido,Shido / Non-Combativity,03:38,https://judobase.ijf.org/#/competition/contest...,Grand Slam Osaka 2018
3,PARK Da Sol,0,Ippon,,03:46,https://judobase.ijf.org/#/competition/contest...,Grand Slam Osaka 2018
4,PUPP Reka,1,Shido,Shido / Non-Combativity,01:22,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018


__you__ = 1 significa que es tuyo (el shido, wazari...)  
__you__ = 0 significa que es del oponente

In [68]:
df.groupby(['you', 'action']).size()

you  action                
0    Cancel Waza-ari            2
     HSK (3rd shido)            2
     Ippon                      4
     Shido                     22
     Waza-Ari-awasete-ippon     2
     Waza-ari                  16
1    Ippon                      3
     Shido                     21
     Waza-Ari-awasete-ippon     1
     Waza-ari                   9
dtype: int64

### Detail Estrella

In [69]:
df.groupby(['you', 'action_detail']).size()

you  action_detail                    
0                                          4
     Ashi-waza / Hiza-guruma               1
     Ashi-waza / O-soto-gaeshi             1
     Ashi-waza / O-soto-gari               2
     Ashi-waza / O-uchi-gaeshi             1
     Ashi-waza / O-uchi-gari               3
     Ashi-waza / Okuri-ashi-harai          1
     Ashi-waza / Uchi-mata                 1
     Cancel Yuko                           1
     Ma-sutemi-waza / Sumi-gaeshi          1
     Ma-sutemi-waza / Tomoe-nage           1
     Osaekomi-waza / Kesa-gatame           1
     Shido / Avoid-Grip                    4
     Shido / Defensive-Posture             4
     Shido / False-Attack                  4
     Shido / Hold-Sleeve-Ends              2
     Shido / Hold-Trouser-Leg              1
     Shido / Non-Combativity               7
     Shido / Outside-Contest-Area          2
     Te-waza / Ippon-seoi-nage             1
     Te-waza / Seoi-nage                   2
     Te-waza / S

### Battles Gaitero

In [70]:
# dataframe containing battles info
df = conn.as_pandas('select * from gaite')

In [72]:
# ver ejemplo
df.head(5)

,opponent,you,action,action_detail,time,url_youtube,event
0,SHMAILOV Baruch,1,Shido,Shido / False-Attack,00:22,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
1,SHMAILOV Baruch,0,Shido,Shido / False-Attack,01:28,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
2,SHMAILOV Baruch,1,Shido,Shido / Hold-Trouser-Leg,03:55,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
3,SHMAILOV Baruch,0,Ippon,Ashi-waza / Uchi-mata,04:23,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
4,BAI Zhijie,1,Shido,Shido / Avoid-Grip,01:34,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018


__you__ = 1 significa que es tuyo (el shido, wazari...)  
__you__ = 0 significa que es del oponente

In [73]:
df.groupby(['you', 'action']).size()

you  action         
0    Cancel Shido        1
     HSK (3rd shido)     5
     Ippon               6
     Shido              24
     Waza-ari            6
1    Cancel Ippon        2
     Cancel Waza-ari     1
     HSK (3rd shido)     4
     Ippon               1
     Shido              38
     Waza-ari           11
dtype: int64

### Detail Gaitero

In [74]:
df.groupby(['you', 'action_detail']).size()

you  action_detail                     
0    Ashi-waza / Ko-uchi-gari               1
     Ashi-waza / O-uchi-gari                2
     Ashi-waza / Uchi-mata                  1
     Koshi-waza / Sode-tsurikomi-goshi      1
     Koshi-waza / Tsuri-goshi               1
     Osaekomi-waza / Kesa-gatame            1
     Osaekomi-waza / Ura-gatame             1
     Osaekomi-waza / Yoko-shiho-gatame      1
     Shido / Avoid-Grip                     1
     Shido / Defensive-Posture              2
     Shido / False-Attack                  11
     Shido / Hold-Same-Side                 1
     Shido / Non-Combativity               11
     Shido / Outside-Contest-Area           4
     Te-waza / Seoi-nage                    1
     Te-waza / Tai-otoshi                   1
     Yoko-sutemi-waza / O-soto-makikomi     1
1                                           4
     Ashi-waza / O-uchi-gaeshi              1
     Ashi-waza / Uchi-mata-gaeshi           1
     Koshi-waza / Sode-tsurikomi-goshi  

### Battles Competidor Obtener

In [15]:
driver = load_chromedriver(en_casa=True)

In [16]:
nombre = 'GAITERO MARTIN Alberto'

In [18]:
conn = SQLiteConnection('../judo.sql')

battles_comp = conn.as_pandas(f'select * from battles where local=="{nombre}"', parse_dates=['date'], index_col='index').reset_index(drop=True)

battles_comp.head()

,wins,local,opponent,event,date,round,local_points,opponent_points,duration,category,has_video,competition_family,url_video
0,0,GAITERO MARTIN Alberto,SHMAILOV Baruch,Grand Prix The Hague 2018,2018-11-16,Round 3,0,100,04:23,men_66,1,Grand Prix,https://judobase.ijf.org/#/competition/contest...
1,1,GAITERO MARTIN Alberto,BAI ZHIJIE,Grand Prix The Hague 2018,2018-11-16,Round 2,10,0,04:00,men_66,1,Grand Prix,https://judobase.ijf.org/#/competition/contest...
2,1,GAITERO MARTIN Alberto,GAFUROV Mirzoiusuf,European Championships Gyor 2018,2018-11-02,Final,10,0,00:00,men_66,1,Continental Championships,https://judobase.ijf.org/#/competition/contest...
3,1,GAITERO MARTIN Alberto,NAGLIASHVILI Tornike,European Championships Gyor 2018,2018-11-02,Semi-Final,100,0,03:36,men_66,1,Continental Championships,https://judobase.ijf.org/#/competition/contest...
4,1,GAITERO MARTIN Alberto,BALKAROV Alim,European Championships Gyor 2018,2018-11-02,Quarter-Final,10,0,05:00,men_66,1,Continental Championships,https://judobase.ijf.org/#/competition/contest...


In [19]:
all_actions = []

for battle in battles_comp.itertuples():
    print(battle.opponent)
    
    if battle.url_video is None:
        continue
    else:
        actions_battle = get_info_video_judobase(battle.local, driver, battle.url_video)
        actions_battle = (a + (battle.event,) for a in actions_battle)
        all_actions.extend(actions_battle)

SHMAILOV Baruch
BAI ZHIJIE
GAFUROV Mirzoiusuf
NAGLIASHVILI Tornike
BALKAROV Alim
BINHAK Tomas
SHERSHAN Dzmitry
SERIKZHANOV Yerlan
ZHUMAKANOV Yeldos
VIERU Denis
BOUSHITA Abderrahmane
SCHEIBEL Manuel
CHIBANA Charles
BURNS Nathon
RAMIREZ RAMOS David
URIARTE Gar
ZONIS Eyal
HOFAECKER Sebastian
PIRAS Matteo
OLEINIC Sergiu
SHMAILOV Baruch
ZANTARAIA Georgii
MARIAC Alexandre
ABDULZHALILOV Abdula
CRISOSTOMO Joao
SERIKZHANOV Yerlan
AZEMA Kevin
VAN GANSBEKE Kenneth
SHAMILOV Yakub
KHAMETOV Islam
VAN HARTEN Matthijs
SANDAL Sinan
OLEINIC Sergiu
BASSOU Imad
FLICKER Tal
SHMAILOV Baruch
JEREB Andraz
HERNANDEZ Angel
KHACHATRIAN Hevorh
URIARTE Gar
STEFANELLI Biagio
RASKOPINE Alexander
GORJANACZ Zsolt
MINKOU Dzmitry
WAWRZYCZEK Patryk
IADOV Bogdan
TIEFGRABER Andreas
LEFEVERE Jasper
ZACHEJA Maciej
HASANAGAEV Robert


In [20]:
df = pd.DataFrame(all_actions, columns=('opponent', 'you', 'action', 'action_detail', 'time', 'url_youtube', 'event'))
df.head()

,opponent,you,action,action2,time,url_youtube,event
0,SHMAILOV Baruch,1,Shido,Shido / False-Attack,00:22,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
1,SHMAILOV Baruch,0,Shido,Shido / False-Attack,01:28,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
2,SHMAILOV Baruch,1,Shido,Shido / Hold-Trouser-Leg,03:55,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
3,SHMAILOV Baruch,0,Ippon,Ashi-waza / Uchi-mata,04:23,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018
4,BAI Zhijie,1,Shido,Shido / Avoid-Grip,01:34,https://judobase.ijf.org/#/competition/contest...,Grand Prix The Hague 2018


In [24]:
nombre_tabla = 'gaite'

In [25]:
if nombre_tabla == 'estre':
    print('CUIDAO')
else:
    conn.add_table(nombre_tabla, df, index=False)

# Más

In [10]:
videos_estre = conn.as_pandas('select * from videos_info where local like "%SHERIFF%"')

In [11]:
videos_estre

,url_video,url_youtube,local,opponent
0,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=8b7mZQsTmK0,LOPEZ SHERIFF Estrella,PARK Da Sol
1,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=HdqymnSiloc,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
2,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=JgPOEHOJ2jo,LOPEZ SHERIFF Estrella,GNETO Astride
3,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=xpd4OZFzESk,LOPEZ SHERIFF Estrella,TSCHOPP Evelyne
4,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=FJmiFvoi8Ro,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
5,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=RfaQdGW0NwY,LOPEZ SHERIFF Estrella,KOSE Mai
6,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=U00rk9umAkY,LOPEZ SHERIFF Estrella,KRASNIQI Distria
7,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=kLIefZ6V2L4,LOPEZ SHERIFF Estrella,KUTSENKO Aigul
8,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=WSa7eH5Vv9U,LOPEZ SHERIFF Estrella,THUMM Verena
9,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=hJaATsCUcwM,LOPEZ SHERIFF Estrella,BEZALEL Ariel


### Filter and extract some videos

problema: cuando falla uno salta una pestaña y jode el resto...

In [8]:
some_videos = all_battles[all_battles.local=='WADA Rinoko'].url_video

for i, url_video in enumerate(some_videos):
    if i % 20 == -1:
        print(i)
    
    if url_video is None:
        continue
    # else
    
    try:
        local, opponent, url_youtube = get_info_video_judobase(driver, url_video)
    except:
        local, opponent, url_youtube = ('error',) * 3
        
    # assert 'https://www.youtube.com/watch?' in url_youtube, f'Strange url_youtube: {url_youtube}'

    conn.query(
        f'''
        INSERT OR IGNORE INTO videos_info 
        (url_video, url_youtube, local, opponent) 
        VALUES
        ("{url_video+'bb'}", "{url_youtube}", "{local}", "{opponent}");
        '''
    )

In [11]:
videos_estre

,url_video,url_youtube,local,opponent
0,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=8b7mZQsTmK0,LOPEZ SHERIFF Estrella,PARK Da Sol
1,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=HdqymnSiloc,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
2,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=JgPOEHOJ2jo,LOPEZ SHERIFF Estrella,GNETO Astride
3,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=xpd4OZFzESk,LOPEZ SHERIFF Estrella,TSCHOPP Evelyne
4,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=FJmiFvoi8Ro,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
5,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=RfaQdGW0NwY,LOPEZ SHERIFF Estrella,KOSE Mai
6,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=U00rk9umAkY,LOPEZ SHERIFF Estrella,KRASNIQI Distria
7,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=kLIefZ6V2L4,LOPEZ SHERIFF Estrella,KUTSENKO Aigul
8,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=WSa7eH5Vv9U,LOPEZ SHERIFF Estrella,THUMM Verena
9,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=hJaATsCUcwM,LOPEZ SHERIFF Estrella,BEZALEL Ariel


### Do all video_urls remaining

In [57]:
proposed_videos = all_battles.loc[all_battles.category == 'men_73', 'url_video']

In [58]:
proposed_videos

Series([], Name: url_video, dtype: object)

In [20]:
all_videos = conn.as_pandas('select * from videos_info')

In [23]:
for v in all_videos.url_video:
    print(v)

NameError: name 'all_videos' is not defined

In [32]:
all_battles.url_video.values.isin(all_videos.url_video.values).sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [29]:
all_videos.url_video.isin(all_battles.url_video)

0    False
1    False
Name: url_video, dtype: bool

In [24]:
all_battles.url_video.isin(all_videos.url_video).sum()

0

In [35]:
all_videos

NameError: name 'all_videos' is not defined

### Extraer info y acceder al youtube

### Extraer info

hay que juntarlo con extraccion url

In [38]:
videos_estre.url_video.str.slice(-2,)

0     bb
1     bb
2     bb
3     bb
4     bb
5     bb
6     bb
7     bb
8     bb
9     bb
10    bb
11    bb
12    bb
Name: url_video, dtype: object

In [32]:
urll = videos_estre.url_video.iloc[1]

In [39]:
urll = urll[:-2]

In [41]:
driver.get(urll)

In [42]:
# tabla con puntuaciones etiquetadas en tiempo
tabla_timed_points = driver.find_elements_by_tag_name('tbody')[1]

In [43]:
filas = tabla_timed_points.find_elements_by_tag_name('tr')

In [44]:
nrows = len(filas[0].find_elements_by_tag_name('td'))
nrows

3

In [45]:
if nrows == 2:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

else:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

    print(f'{nrows} rows!')

0 0 Waza-ari
0 1 00:09
0 2 
1 0 
1 1 02:36
1 2 Shido
Shido / Non-Combativity
3 rows!


In [46]:
seconds = 22

In [47]:
url_youtube_time = url_youtube + '&t=' + str(seconds)
url_youtube_time

'https://www.youtube.com/watch?v=3ajXhPeC9dQ&t=22'

In [48]:
driver.get(url_youtube_time)

### Youtube downloads

In [49]:
url_youtube = videos_estre.iloc[0,1]

In [50]:
url_youtube

'https://www.youtube.com/watch?v=8b7mZQsTmK0'

In [51]:
from pytube import YouTube

In [52]:
y = YouTube(url_youtube)

In [53]:
video = y.streams.filter(res='720p', mime_type='video/mp4').first()

In [54]:
assert video is not None, 'No video'

In [55]:
%time video.download(filename='estre720')

CPU times: user 2.97 s, sys: 2.28 s, total: 5.25 s
Wall time: 9.88 s


'/home/sheriff/Documents/projects/judo/judo/estre720.mp4'

### Edición con moviepy

In [35]:
3

3

In [56]:
import requests

In [3]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips

In [2]:
text0 = TextClip("Primer chunk", font='Bookman-Demi', fontsize=70, color='red').set_position(('right', 'top'), relative=True).set_duration(5)


[MoviePy] This command returned an error !

OSError: MoviePy Error: creation of None failed because of the following error:

convert: delegate library support not built-in '/usr/share/fonts/type1/gsfonts/b018015l.pfb' (Freetype) @ warning/annotate.c/RenderFreetype/1797.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/556.
convert: no images defined `PNG32:/tmp/tmp6dqv06yt.png' @ error/convert.c/ConvertImageCommand/3300.
.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

In [3]:
text1 = TextClip("Segundo chunk", fontsize=70, color='blue').set_position((0.4, 0), relative=True).set_duration(5)
# text2 = TextClip("Tercer chunk", fontsize=70, color='green').set_position((0.8, 0), relative=True).set_duration(5)

# textos = (text0, text1, text2)

SyntaxError: invalid syntax (<ipython-input-3-f2f427ef38e1>, line 1)

In [2]:
clip = VideoFileClip('./estre720.mp4')

In [3]:
clip.duration

353.85

In [4]:
corte = clip.subclip(40,50)

In [64]:
import pkg_resources

In [65]:
pkg_resources.get_distribution('moviepy')

moviepy 0.2.3.5 (/home/sheriff/anaconda3/envs/video/lib/python3.7/site-packages)

In [63]:
3+3

6

In [13]:
textos = concatenate_videoclips([text0, text1])

In [15]:
final_clip = CompositeVideoClip([corte, textos])

In [16]:
final_clip.duration

4

In [17]:
final_clip.write_videofile("my_concatenatio2n2.mp4")

[MoviePy] >>>> Building video my_concatenatio2n2.mp4
[MoviePy] Writing audio in my_concatenatio2n2TEMP_MPY_wvf_snd.mp3


100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 271.76it/s]


[MoviePy] Done.
[MoviePy] Writing video my_concatenatio2n2.mp4


 99%|███████████████████████████████████████████████████████████████████████████████▏| 100/101 [00:06<00:00, 12.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_concatenatio2n2.mp4 

